In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import pickle
from __future__ import division
import csv
import tpm_identification
from auxiliar_functions import *
import datetime

In [4]:
dict_metro = load_metro_dictionary()

In [7]:
if os.name == 'nt':
    frame_path = 'C:\Users\catalina\Documents\Datois\etapas_2013_abril_allyearsids_10_100000.csv'
    df_period_path = 'C:\Users\catalina\Documents\Datois\etapas_2013_septiembre_allyearsids_10_100000.csv'
else:
    frame_path = '/home/cata/Documentos/Datois/etapas_2013_abril_allyearsids_10_100000.csv'
    df_period_path = 'C:\Users\catalina\Documents\Datois\etapas_2013_septiembre_allyearsids_10_100000.csv'

In [9]:
frame = pd.read_csv(frame_path)
frame['tiempo_subida'] = pd.to_datetime(frame.tiempo_subida)
frame = frame.sort_values(by=['id', 'tiempo_subida'])
frame.head()

,tiempo_subida,id,x_subida,y_subida,tipo_transporte,serviciosentidovariante,tipo_dia,nviaje,netapa,x_bajada,y_bajada,tiempo_bajada,par_subida,par_bajada,zona_subida,zona_bajada,adulto
23,2013-04-14 06:45:44,1132106,348108,6289153,BUS,T203 00R,DOMINGO,1,1,346818,6299394,2013-04-14 07:07:02,T-22-205-SN-65,E-20-190-SN-40,328,307,0
22,2013-04-14 07:51:52,1132106,346751,6299389,BUS,T502 00I,DOMINGO,2,1,351363,6302549,2013-04-14 08:04:11,E-20-291-PO-20,T-15-135-PO-5,307,188,0
21,2013-04-14 19:56:47,1132106,351368,6302559,BUS,T502 00R,DOMINGO,3,1,346763,6299568,2013-04-14 20:09:11,T-15-135-OP-110,T-4-19-NS-100,188,55,0
20,2013-04-14 20:15:25,1132106,346713,6299427,BUS,T203 00I,DOMINGO,3,2,348095,6289148,2013-04-14 20:40:51,E-20-199-NS-2,T-24-205-NS-20,307,348,0
19,2013-04-15 21:04:59,1132106,348103,6289191,BUS,T206 00R,LABORAL,4,1,346844,6299320,2013-04-15 21:33:23,T-22-205-SN-65,T-20-190-SN-35,328,309,0


In [11]:
df_id_period = pd.read_csv(df_period_path)
df_id_period['tiempo_subida'] = pd.to_datetime(df_id_period.tiempo_subida)
df_id_period = df_id_period.sort_values(by=['id', 'tiempo_subida'])

In [12]:
reload(tpm_identification)

<module 'tpm_identification' from 'tpm_identification.py'>

In [13]:
profiles = tpm_identification.get_spatiotemporal_profiles(frame['id'],frame['tiempo_subida'],frame['par_subida'],frame['par_bajada'])

In [19]:
sequences = tpm_identification.get_spatiotemporal_sequences(df_id_period['id'],df_id_period['tiempo_subida'],df_id_period['par_subida'],df_id_period['par_bajada'])

In [ ]:
start_time = time.time()
iden = tpm_identification.get_spatiotemporal_identification_matrix(profiles,sequences)
delta_time = time.time() - start_time


In [ ]:
with open('iden_matrix_spatiotemporal.pickle','w') as f:
    pickle.dump(iden,f)

In [ ]:
iden_matrix = np.matrix(iden)
df_ident = pd.DataFrame(iden_matrix)
i = 0
identified_indexs = []
wrong_indexs = []
correct_indexs = []
selected_indexs = []
n_identified = 0
limit = min(len(users_profiles),len(profiles))
while (i<limit):
    the_index = np.argmax(iden_matrix[:,i])
    selected_indexs.append(np.max(iden_matrix[:,i]))
    identified_indexs.append(the_index)
    if(the_index!=i):
        wrong_indexs.append(the_index)
    else:
        correct_indexs.append(the_index)
        n_identified += 1
    i += 1

In [ ]:
porcentaje_correcto = n_identified*100/limit
print str(round(porcentaje_correcto,2))+ "%"